In [ ]:
# Imports
!pip install cpmpy numpy --quiet

import cpmpy as cp
import numpy as np
import time

def solve_with_time(model, time_limit=None):

    start_time = time.time()
    solution_found = model.solve(time_limit=time_limit)
    end_time = time.time()

    elapsed_time = end_time - start_time
    print("Time elapsed: {:.4f} seconds".format(elapsed_time))

    return solution_found

**Useful Resources:**
* CPMpy summary sheet: https://cpmpy.readthedocs.io/en/latest/summary.html
* CPMpy modeling documentation: https://cpmpy.readthedocs.io/en/latest/modeling.html

## **Session 4: Advanced Modelling**

### Outline:
1. Viewpoints
2. Channelling
3. Auxiliary Variables
4. Implied Constraints
5. Extra Exercises

## **1. Viewpoints**

In constraint programming, we often have multiple ways to represent the same problem. These representations, or "viewpoints," affect both how the CP solver explores the solution space and how easy it is to formulate the constraints. A carefully chosen viewpoint can:
- Reduce the complexity of the model, making it **faster** to solve
- Make the constraints **easier** to express
- **Implicitly** model constraints as part of the variables/domains definitions
- Improve **readability**

---

Consider the following problem:

We have $n$ pigeons and $m$ pigeonholes. Each pigeon must be assigned to one pigeonhole, but each pigeonhole can hold at most one pigeon.
Here, we consider $n=50$ pigeons and $m=60$ pigeonholes.

The output can be of the form:
```
Pigeon 1 goes to hole 1
Pigeon 2 goes to hole 7
...
Pigeon 50 goes to hole 53
```

Modelling this problem is quite straightforward. The focus is on constructing two models for this problem using **different viewpoints**.

### Viewpoint 1: Pigeons as variables

<details>
  <summary>Click to reveal hints</summary>

* We need a decision variable for each pigeon representing which hole it goes into.
* The [AllDifferent](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.AllDifferent) constraint fits nicely when the pigeons are selected as decision variables.
</details>

In [ ]:
num_pigeons = 50
num_pigeonholes = 60

# TODO: Decision variables
pigeons = None

# Constraints
model_vp1 = cp.Model()

# TODO: Constraints
model_vp1.add(None)

# Solve the model
solved = solve_with_time(model_vp1)

if solved:
  print(model_vp1.status())
  # TODO: Print the solution

else:
  print("No solution found.")

### Viewpoint 2: Pigeonholes as variables

<details>
  <summary>Click to reveal hints</summary>

- Make sure that the domain includes also the option that the hole may be empty
- The constraint that each pigeon must be assigned to exactly one pigeonhole can be expressed in various ways, some of them are:
  - For each pigeon use `cp.sum` and ensure that it is equal to 1.
  - Use [GlobalCardinalityCount](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.GlobalCardinalityCount)
  - Use [AllDifferentExcept0](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalconstraints.html#cpmpy.expressions.globalconstraints.AllDifferentExcept0) and [Count](https://cpmpy.readthedocs.io/en/latest/api/expressions/globalfunctions.html#cpmpy.expressions.globalfunctions.Count)
</details>

In [ ]:
# Decision variables
pigeonholes = None

# Constraints
model_vp2 = cp.Model()

# TODO: Model the problem and print the solution.


**Observations:**
1. Which viewpoint was easier to model?
2. Which viewpoint was faster to solve?
3. Which constraints are implicitly covered in each viewpoint?

In [ ]:
# TODO: Count the time taken for model_vp1.solve() and model_vp2.solve()


## **2. Channeling**

Some constraints are more naturally expressed in one viewpoint, while other constraints of the same model are maybe more naturally expressed in another viewpoint. In such cases, **channeling** is a useful technique that connects variables from different viewpoints within a single model. Some key benefits:

1. You can use the most intuitive viewpoint for each constraint.
2. The solver may deduce information across linked viewpoints, potentially speeding up the solving process.
3. The final models can become easier to read, maintain, and extend.

Channeling is achieved by adding constraints that ensure consistency between the decision variables of different viewpoints.

---

Consider the *Student Seating Problem* from the lecture:

You are tasked with finding an optimal seating arrangement for `nStudents = 15` students across `nTables = 5` tables, where there are `nChairs = 20` chairs in total. You are given a list of chair assignments for each table:
`Chairs = [1..4, 5..8, 9..12, 13..16, 17..20]`.
The goal is to assign **all** students to the chairs in such a way that each table has either at least half or no chairs occupied.

Create variables with two different viewpoints and try to model each constraint with the viewpoint that is easier. Before solving, it is necessary that the variables between the viewpoints are logically connected.

<details>
  <summary>Click to reveal hints</summary>

There are two high-level constraints that need to be satisfied:
1. From a student perspective: **All** students sit on a **different** chair
2. From a chair perspective: In each table, either **all chairs are empty** or **at least half are non-empty**.
</details>

In [ ]:
# Data
n_students = 15
n_chairs = 20
n_tables = 5

# Chair indexes for each table
# [1..4, 5..8, 9..12, 13..16, 17..20]
chairs_per_table = n_chairs // n_tables
tables = [list(range(start, start + chairs_per_table))
          for start in range(1, n_chairs + 1, chairs_per_table)]

# Model
model = cp.Model()

# Viewpoint 1: Students as vars
students = None  # TODO

# TODO: Add the constraint(s) that is (are) easier to express with vp1

# Viewpoint 2: Chairs as vars
chairs = None  # TODO (make sure you also include a value for the empty chair)

# TODO: Add the constraint(s) that is (are) easier to express with vp2


# Channeling: At this point we have modeled half of the constraints with the
# vars of vp1 and half with those of vp2. We need to merge the two viewpoints.

# TODO: Connect the students and chairs viewpoints


# TODO: Solve the model and print the solution


## **3. Auxiliary Variables**

Auxiliary variables can help in expressing complex constraints more naturally and clearly. They can be useful when establishing relationships between decision variables and additional properties or constraints, making it easier to formulate constraints. So, they are used:

* Because it is difficult to express the constraints in terms of the existing variables, or
* To allow the constraints to be expressed in a form that would propagate better.

**Car Sequencing**

Imagine an assembly line producing different types of cars, each with optional features (e.g. air-conditioning, sunroof etc.). Each station on the assembly line handles a specific feature and has a limited capacity, meaning it can only work on a certain percentage of the cars that pass through.

To prevent any station from being overwhelmed, cars must be arranged in a sequence so that the number of cars needing a particular feature never exceeds the station's capacity. For example, if a station can only handle half of the cars that pass by, then at most 1 out of every 2 cars in the sequence should require that feature.

A model of this problem is also available in the slides of lecture 5, page 23.
Try to model it without looking at the slides first.

In [ ]:
# Data
at_most = [1, 2, 2, 2, 1]  # The maximum number of times an option can be present in a group of consecutive timeslots (see 'per_slots' below)
per_slots = [2, 3, 3, 5, 5]  # The number of consecutive timeslots (window) in which an option has a limit (see 'at_most' above)

demand = [1, 1, 2, 2, 2, 2]  # The demand per type of car
requires = cp.cpm_array([
    [1, 0, 1, 1, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 1],
    [0, 1, 0, 1, 0],
    [1, 0, 1, 0, 0],
    [1, 1, 0, 0, 0]])  # Which options (columns) are needed for each car type (rows).

n_cars = sum(demand)  # The amount of cars to sequence
n_options = len(at_most)  # The amount of different options
n_types = len(demand)  # The amount of different car types

# Decision Variables
sequence = None  # TODO: The sequence of cars

# TODO: Define the auxiliary variables -> Which options are needed for each car in the sequence.
option = None

# Model
car_seq_model = cp.Model()

# TODO: The amount of each type of car in the sequence has to be equal to the demand for that type

# TODO: Make sure that the options in the option table correspond to those of the car type

# TODO: Check that no more than "at most" car options are used per "per_slots" slots


# TODO: Solve and print the sequence of cars and which options are required per car.


For reference, here is a solution of this problem:

```
Sequence:  0 2 5 1 5 3 4 2 3 4
------------------------------
Option 1:  1 0 1 0 1 0 1 0 0 1
Option 2:  0 1 1 0 1 1 0 1 1 0
Option 3:  1 0 0 0 0 0 1 0 0 1
Option 4:  1 0 0 1 0 1 0 0 1 0
Option 5:  0 1 0 0 0 0 0 1 0 0
```

## **4. Implied Constraints**

As taken from the lecture slides:

Can we do something to make our model quicker?
During the search process, the solver will explore several infeasible parts of the search tree. Can we avoid that?
Although implied constraints are logically redundant, and do not change the set of solutions, they can be used to reduce the search effort of the solving process.

Consider the **Magic Series problem**, also seen at lecture 5, page 29:

The element at index i in I = 0..(n-1) is the number of occurrences of i, for an integer n. Create a model that finds a magic series for n = 100.

Print the time taken to solve this model.

In [ ]:
n = 100

# TODO: Decision Variables
Magic = None

# Model
model = cp.Model()

# TODO: Add the constraint


# TODO: Solve the model and print the solution (print the time taken)


We can significantly improve solving time by adding implied constraints.

Add implied constraints (try it initially without looking at them in the slides), and compare the time taken to solve the new model with the above one.

In [ ]:
# TODO: Implied constraints



# TODO: Solve the model and print the solution (print the time taken)


## **5. Extra Exercises**


### Auxiliary Variables: Orthogonal Latin Squares

Consider the [Orthogonal Latin Squares](https://en.wikipedia.org/wiki/Mutually_orthogonal_Latin_squares) problem:

A Latin square is an $ n \times n $ array filled with $n$ different symbols, each occurring exactly once in each row and exactly once in each column. In other words, a Latin Square is a square grid where:

* Each row contains all $n$ distinct symbols exactly once.
* Each column contains all $n$ distinct symbols exactly once.
* No symbol appears twice in the same row or column.

Two Latin squares are said to be orthogonal if, when placed on top of each other, each possible combination of their symbols appears exactly once.

**Instructions:**

You have a set of $ n $ integers (0..n-1), and your task is to construct two $ n \times n $ Latin squares that are orthogonal.

1. Define decision variables for the Latin squares.
2. Introduce auxiliary variables to represent each cell's pairing from both Latin squares.
3. Ensure that each pairing is unique across the entire grid.

In [ ]:
n = 7  # Size of the Latin square, and number of symbols

# TODO: Decision variables
square1 = None
square2 = None

# TODO: Auxiliary variables to combine symbols from both squares
pairing = None

# Model
model = cp.Model()

# TODO: Latin square constraints

# TODO: Connect the pairing variables with the square variables

# TODO: All pairings should be unique


# TODO: Solve the model and print the squares


Now, to see the benefit of auxiliary variables, try to model the same problem but without using `pairings`. The orthogonality constraint needs to be formulated directly over the square variables. This is more complicated to model, and possibly also slower for the solver (check the run time).

In [ ]:
n = 7  # Size of the Latin square, and number of symbols

# Decision variables for two Latin squares
square1 = None
square2 = None

# Model
model_no_aux = cp.Model()

# TODO: Latin square constraints

# TODO: Orthogonality constraint: Ensure each combination is unique by pair-wise comparing all pairs


# TODO: Solve the model and print the time taken. Print the time taken also for the previous model that contains auxiliary variables.


### Implied Constraints: Car Sequencing

Consider again the Car Sequencing problem. Add the implied constraints that are recommended in the slides of lecture 5, page 27.

For more details on the implied constraints of Car Sequencing, see this [article](https://www.researchgate.net/publication/220838036_Solving_the_Car_Sequencing_Problem_in_Constraint_Logic_Programming#fullTextFileContent), Pages 8-9.

In [ ]:
# TODO: Implied Constraints
car_seq_model.add(None)